In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [3]:
data_folder = './data'
df = pd.read_csv(os.path.join(data_folder, 'intermediate.csv'), index_col=0)
print(df.shape)
df.head()

C:\Users\bruno\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(3311925, 109)


,no_municipio_residencia,sg_uf_residencia,nu_idade,tp_sexo,tp_estado_civil,tp_cor_raca,tp_nacionalidade,no_municipio_nascimento,sg_uf_nascimento,tp_st_conclusao,tp_ano_concluiu,tp_escola,tp_ensino,in_treineiro,co_escola,no_municipio_esc,sg_uf_esc,tp_dependencia_adm_esc,tp_localizacao_esc,tp_sit_func_esc,in_baixa_visao,in_cegueira,in_surdez,in_deficiencia_auditiva,in_surdo_cegueira,in_deficiencia_fisica,in_deficiencia_mental,in_deficit_atencao,in_dislexia,in_discalculia,in_autismo,in_visao_monocular,in_outra_def,in_gestante,in_lactante,in_idoso,in_estuda_classe_hospitalar,in_sem_recurso,in_braille,in_ampliada_24,in_ampliada_18,in_ledor,in_acesso,in_transcricao,in_libras,in_tempo_adicional,in_leitura_labial,in_mesa_cadeira_rodas,in_mesa_cadeira_separada,in_apoio_perna,in_guia_interprete,in_computador,in_cadeira_especial,in_cadeira_canhoto,in_cadeira_acolchoada,in_prova_deitado,in_mobiliario_obeso,in_lamina_overlay,in_protetor_auricular,in_medidor_glicose,in_maquina_braile,in_soroban,in_marca_passo,in_sonda,in_medicamentos,in_sala_individual,in_sala_especial,in_sala_acompanhante,in_mobiliario_especifico,in_material_especifico,in_nome_social,no_municipio_prova,sg_uf_prova,tp_presenca_cn,tp_presenca_ch,tp_presenca_lc,tp_presenca_mt,nu_nota_cn,nu_nota_ch,nu_nota_lc,nu_nota_mt,tp_lingua,tp_status_redacao,nu_nota_redacao,q001,q002,q003,q004,q005,q006,q007,q008,q009,q010,q011,q012,q013,q014,q015,q016,q017,q018,q019,q020,q021,q022,q023,q024,q025
0,Ariquemes,RO,31.0,Feminino,Não informado,Branca,Brasileiro(a),Aracruz,ES,Já concluí o Ensino Médio,2009,Não respondeu,Ensino Regular,Não,NaN,NaN,NaN,NaN,NaN,NaN,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Ariquemes,RO,Presente na prova,Presente na prova,Presente na prova,Presente na prova,426.5,487.4,530.9,415.3,Espanhol,Sem problemas,680.0,Não completou a 4ª série/5º ano do Ensino Fund...,Nunca estudou.,"Grupo 1: Lavrador, agricultor sem empregados, ...","Grupo 1: Lavrador, agricultor sem empregados, ...",2,"De R$ 1.996,01 até R$ 2.495,00.",Não.,"Sim, um.","Sim, dois.","Sim, dois.","Sim, dois.","Sim, um.","Sim, um.","Sim, um.","Sim, um.","Sim, um.",Não.,Não.,"Sim, um.",Não.,Não.,"Sim, dois.",Não.,"Sim, um.",Sim
1,Belo Horizonte,MG,23.0,Feminino,Solteiro(a),Parda,Brasileiro(a),Belo Horizonte,MG,Já concluí o Ensino Médio,2016,Não respondeu,NaN,Não,NaN,NaN,NaN,NaN,NaN,NaN,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Belo Horizonte,MG,Presente na prova,Presente na prova,Presente na prova,Presente na prova,463.0,452.3,456.6,407.2,Inglês,Sem problemas,520.0,Não completou a 4ª série/5º ano do Ensino Fund...,"Completou a 4ª série/5º ano, mas não completou...",Grupo 4: Professor (de ensino fundamental ou m...,"Grupo 2: Diarista, empregado doméstico, cuidad...",2,"De R$ 1.996,01 até R$ 2.495,00.",Não.,"Sim, um.","Sim, dois.",Não.,Não.,"Sim, um.",Não.,"Sim, um.",Não.,Não.,Não.,Não.,"Sim, um.",Não.,Não.,"Sim, dois.",Não.,"Sim, um.",Sim
2,Manhuaçu,MG,17.0,Masculino,Não informado,Parda,Brasileiro(a),Belo Horizonte,MG,Estou cursando e concluirei o Ensino Médio no ...,Não informado,Pública,Ensino Regular,Não,31075167.0,Manhuaçu,MG,Estadual,Urbana,Em atividade,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Manhuaçu,MG,Presente na prova,Presente na prova,Presente na prova,Presente na prova,381.7,474.4,530.6,622.8,Inglês,Sem problemas,600.0,"Completou o Ensino Médio, mas não completou a ...",Não completou a 4ª série/5º ano do Ensino Fund...,"Grupo 3: Padeiro, cozinheiro industrial ou em ...",Grupo 4: Professor (de ensino fundamental ou m...,3,"

In [4]:
df = df.drop(columns='co_escola')

In [5]:
obj_columns = df.select_dtypes('object').columns
df[obj_columns] = df[obj_columns].astype('category')

Missing data

In [6]:
print(df['nu_idade'].median())
df['nu_idade'] = df['nu_idade'].fillna(df['nu_idade'].median())

19.0


In [7]:
df = df.dropna(subset=['nu_nota_lc', 'nu_nota_ch', 'nu_nota_cn', 'nu_nota_mt', 'nu_nota_redacao'], how='any')
df.shape

(2405748, 108)

In [8]:
cat_columns = df.select_dtypes('category').columns
for cat in cat_columns:
    try:
        df[cat] = df[cat].cat.add_categories('Não informado')
    except ValueError:
        pass
    df[cat] = df[cat].fillna('Não informado')
df.columns[df.isna().sum() > 0]

Index([], dtype='object')

Baseline

In [9]:
feature_columns = [
    'nu_idade', # unicas features numericas
    'q005', # unicas features numericas
    'sg_uf_residencia',
    'tp_sexo',
    'tp_cor_raca',
    'tp_st_conclusao',
    'tp_escola',
    'in_treineiro',
    'tp_dependencia_adm_esc',
    'tp_lingua',
    'q001',
    'q002',
    'q003',
    'q004',
    'q006',
    'q025'
]

target_columns = [
    'nu_nota_lc',
    'nu_nota_ch',
    'nu_nota_cn',
    'nu_nota_mt',
    'nu_nota_redacao'
]

In [10]:
df = pd.get_dummies(df, columns=feature_columns[2:], drop_first=True)
df.shape

(2405748, 191)

In [11]:
df.columns.tolist()

['no_municipio_residencia',
 'nu_idade',
 'tp_estado_civil',
 'tp_nacionalidade',
 'no_municipio_nascimento',
 'sg_uf_nascimento',
 'tp_ano_concluiu',
 'tp_ensino',
 'no_municipio_esc',
 'sg_uf_esc',
 'tp_localizacao_esc',
 'tp_sit_func_esc',
 'in_baixa_visao',
 'in_cegueira',
 'in_surdez',
 'in_deficiencia_auditiva',
 'in_surdo_cegueira',
 'in_deficiencia_fisica',
 'in_deficiencia_mental',
 'in_deficit_atencao',
 'in_dislexia',
 'in_discalculia',
 'in_autismo',
 'in_visao_monocular',
 'in_outra_def',
 'in_gestante',
 'in_lactante',
 'in_idoso',
 'in_estuda_classe_hospitalar',
 'in_sem_recurso',
 'in_braille',
 'in_ampliada_24',
 'in_ampliada_18',
 'in_ledor',
 'in_acesso',
 'in_transcricao',
 'in_libras',
 'in_tempo_adicional',
 'in_leitura_labial',
 'in_mesa_cadeira_rodas',
 'in_mesa_cadeira_separada',
 'in_apoio_perna',
 'in_guia_interprete',
 'in_computador',
 'in_cadeira_especial',
 'in_cadeira_canhoto',
 'in_cadeira_acolchoada',
 'in_prova_deitado',
 'in_mobiliario_obeso',
 'in_l

In [12]:
features = [
    'nu_idade',
    'q005',
    'sg_uf_residencia_AL',
    'sg_uf_residencia_AM',
    'sg_uf_residencia_AP',
    'sg_uf_residencia_BA',
    'sg_uf_residencia_CE',
    'sg_uf_residencia_DF',
    'sg_uf_residencia_ES',
    'sg_uf_residencia_GO',
    'sg_uf_residencia_MA',
    'sg_uf_residencia_MG',
    'sg_uf_residencia_MS',
    'sg_uf_residencia_MT',
    'sg_uf_residencia_PA',
    'sg_uf_residencia_PB',
    'sg_uf_residencia_PE',
    'sg_uf_residencia_PI',
    'sg_uf_residencia_PR',
    'sg_uf_residencia_RJ',
    'sg_uf_residencia_RN',
    'sg_uf_residencia_RO',
    'sg_uf_residencia_RR',
    'sg_uf_residencia_RS',
    'sg_uf_residencia_SC',
    'sg_uf_residencia_SE',
    'sg_uf_residencia_SP',
    'sg_uf_residencia_TO',
    'sg_uf_residencia_Não informado',
    'tp_sexo_Masculino',
    'tp_sexo_Não informado',
    'tp_cor_raca_Branca',
    'tp_cor_raca_Indígena',
    'tp_cor_raca_Não declarado',
    'tp_cor_raca_Parda',
    'tp_cor_raca_Preta',
    'tp_cor_raca_Não informado',
    'tp_st_conclusao_Estou cursando e concluirei o Ensino Médio no ano corrente',
    'tp_st_conclusao_Já concluí o Ensino Médio',
    'tp_st_conclusao_Não concluí e não estou cursando o Ensino Médio',
    'tp_st_conclusao_Não informado',
    'tp_escola_Privada',
    'tp_escola_Pública',
    'tp_escola_Não informado',
    'in_treineiro_Sim',
    'in_treineiro_Não informado',
    'tp_dependencia_adm_esc_Federal',
    'tp_dependencia_adm_esc_Municipal',
    'tp_dependencia_adm_esc_Privada',
    'tp_dependencia_adm_esc_Não informado',
    'tp_lingua_Inglês',
    'tp_lingua_Não informado',
    'q001_Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.',
    'q001_Completou a Faculdade, mas não completou a Pós-graduação.',
    'q001_Completou a Pós-graduação.',
    'q001_Completou o Ensino Médio, mas não completou a Faculdade.',
    'q001_Nunca estudou.',
    'q001_Não completou a 4ª série/5º ano do Ensino Fundamental.',
    'q001_Não sei.',
    'q001_Não informado',
    'q002_Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.',
    'q002_Completou a Faculdade, mas não completou a Pós-graduação.',
    'q002_Completou a Pós-graduação.',
    'q002_Completou o Ensino Médio, mas não completou a Faculdade.',
    'q002_Nunca estudou.',
    'q002_Não completou a 4ª série/5º ano do Ensino Fundamental.',
    'q002_Não sei.',
    'q002_Não informado',
    'q003_Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.',
    'q003_Grupo 3: Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.',
    'q003_Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.',
    'q003_Grupo 5: Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.',
    'q003_Não sei.',
    'q003_Não informado',
    'q004_Grupo 2: Diarista, empregado doméstico, cuidador de idosos, babá, cozinheiro (em casas particulares), motorista particular, jardineiro, faxineiro de empresas e prédios, vigilante, porteiro, carteiro, office-boy, vendedor, caixa, atendente de loja, auxiliar administrativo, recepcionista, servente de pedreiro, repositor de mercadoria.',
    'q004_Grupo 3: Padeiro, cozinheiro industrial ou em restaurantes, sapateiro, costureiro, joalheiro, torneiro mecânico, operador de máquinas, soldador, operário de fábrica, trabalhador da mineração, pedreiro, pintor, eletricista, encanador, motorista, caminhoneiro, taxista.',
    'q004_Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.',
    'q004_Grupo 5: Médico, engenheiro, dentista, psicólogo, economista, advogado, juiz, promotor, defensor, delegado, tenente, capitão, coronel, professor universitário, diretor em empresas públicas ou privadas, político, proprietário de empresas com mais de 10 empregados.',
    'q004_Não sei.',
    'q004_Não informado',
    'q006_De R$ 1.497,01 até R$ 1.996,00.',
    'q006_De R$ 1.996,01 até R$ 2.495,00.',
    'q006_De R$ 11.976,01 até R$ 14.970,00.',
    'q006_De R$ 14.970,01 até R$ 19.960,00.',
    'q006_De R$ 2.495,01 até R$ 2.994,00.',
    'q006_De R$ 2.994,01 até R$ 3.992,00.',
    'q006_De R$ 3.992,01 até R$ 4.990,00.',
    'q006_De R$ 4.990,01 até R$ 5.988,00.',
    'q006_De R$ 5.988,01 até R$ 6.986,00.',
    'q006_De R$ 6.986,01 até R$ 7.984,00.',
    'q006_De R$ 7.984,01 até R$ 8.982,00.',
    'q006_De R$ 8.982,01 até R$ 9.980,00.',
    'q006_De R$ 9.980,01 até R$ 11.976,00.',
    'q006_De R$ 998,01 até R$ 1.497,00.',
    'q006_Mais de R$ 19.960,00.',
    'q006_Nenhuma renda.',
    'q006_Não informado',
    'q025_Sim',
    'q025_Não informado'
]

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target_columns], test_size=0.1, random_state=7)

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline

regressor = make_pipeline(
    StandardScaler(),
    MultiOutputRegressor(
        Lasso(random_state=7),
        n_jobs=-1
    )
)

regressor.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('multioutputregressor',
                 MultiOutputRegressor(estimator=Lasso(random_state=7),
                                      n_jobs=-1))])

In [15]:
y_pred = regressor.predict(X_test)
y_pred.shape

(240575, 5)

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

print('MAE:', mean_absolute_error(y_test, y_pred))
print('R^2:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', sqrt(mean_squared_error(y_test, y_pred)))

MAE: 68.61775942404985
R^2: 0.2590455308534347
MSE: 9322.997251122282
RMSE: 96.55566918168131


In [17]:
ests = regressor.named_steps['multioutputregressor'].estimators_
print([ e.intercept_ for e in ests ])
feature_coefs = pd.DataFrame({
    'lc': ests[0].coef_,
    'ch': ests[1].coef_,
    'cn': ests[2].coef_,
    'mt': ests[3].coef_,
    'redacao': ests[4].coef_
}, index=features)
feature_coefs.sort_values('mt', ascending=False)

[522.3855994417077, 509.8603541148909, 477.88475710716864, 523.1906770036387, 579.7026288430532]


,lc,ch,cn,mt,redacao
tp_sexo_Masculino,-0.863973,1.908172,4.972357,14.361498,-13.292897
tp_lingua_Inglês,8.873209,10.364201,8.901633,11.461846,13.730857
tp_st_conclusao_Já concluí o Ensino Médio,7.487786,9.851164,8.101678,11.184600,26.127868
"q006_Mais de R$ 19.960,00.",2.425767,4.666936,4.373196,9.696765,10.199369
"q006_De R$ 3.992,01 até R$ 4.990,00.",2.568067,4.438483,4.207405,7.891369,10.669852
"q006_De R$ 4.990,01 até R$ 5.988,00.",2.423454,4.241788,4.037139,7.789484,10.350588
tp_dependencia_adm_esc_Federal,3.318790,4.869569,5.288973,7.783254,12.306080
"q006_De R$ 9.980,01 até R$ 11.976,00.",1.996161,3.865830,3.932134,7.752274,8.949191
"q006_De R$ 14.970,01 até R$ 19.960,00.",1.891196,3.731075,3.740616,7.680011,8.499030
"q006_De R$ 11.976,01 até R$ 14.970,00.",1.905474,3.737692,3.734300,7.616081,8.722792


In [18]:
pd.DataFrame(feature_coefs.abs().sum(axis=1).sort_values(ascending=False))

,0
tp_st_conclusao_Já concluí o Ensino Médio,62.753097
tp_lingua_Inglês,53.331746
nu_idade,38.013983
tp_sexo_Masculino,35.398899
tp_dependencia_adm_esc_Federal,33.566666
"q003_Grupo 4: Professor (de ensino fundamental ou médio, idioma, música, artes etc.), técnico (de enfermagem, contabilidade, eletrônica etc.), policial, militar de baixa patente (soldado, cabo, sargento), corretor de imóveis, supervisor, gerente, mestre de obras, pastor, microempresário (proprietário de empresa com menos de 10 empregados), pequeno comerciante, pequeno proprietário de terras, trabalhador autônomo ou por conta própria.",33.280676
"q006_Mais de R$ 19.960,00.",31.362034
"q006_De R$ 3.992,01 até R$ 4.990,00.",29.775175
"q006_De R$ 2.994,01 até R$ 3.992,00.",29.447058
"q006_De R$ 4.990,01 até R$ 5.988,00.",28.842452


Segundo modelo

In [19]:
unimportant_features = [
    'q006_Não informado',
    'tp_dependencia_adm_esc_Não informado',
    'q004_Não informado',
    'q003_Não informado',
    'sg_uf_residencia_GO',
    'sg_uf_residencia_Não informado',
    'tp_sexo_Não informado',
    'tp_cor_raca_Parda',
    'tp_cor_raca_Não informado',
    'tp_st_conclusao_Estou cursando e concluirei o Ensino Médio no ano corrente',
    'tp_st_conclusao_Não informado',
    'tp_escola_Não informado',
    'in_treineiro_Não informado',
    'tp_dependencia_adm_esc_Municipal',
    'tp_lingua_Não informado',
    'q001_Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.',
    'q001_Não informado',
    'q002_Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.',
    'q002_Não informado',
    'q025_Não informado'
]

In [20]:
irrelevant_features = [
    'sg_uf_residencia_AL',
    'sg_uf_residencia_AM',
    'sg_uf_residencia_AP',
    'sg_uf_residencia_BA',
    'sg_uf_residencia_CE',
    'sg_uf_residencia_DF',
    'sg_uf_residencia_ES',
    # 'sg_uf_residencia_GO',
    'sg_uf_residencia_MA',
    'sg_uf_residencia_MG',
    'sg_uf_residencia_MS',
    'sg_uf_residencia_MT',
    'sg_uf_residencia_PA',
    'sg_uf_residencia_PB',
    'sg_uf_residencia_PE',
    'sg_uf_residencia_PI',
    'sg_uf_residencia_PR',
    'sg_uf_residencia_RJ',
    'sg_uf_residencia_RN',
    'sg_uf_residencia_RO',
    'sg_uf_residencia_RR',
    'sg_uf_residencia_RS',
    'sg_uf_residencia_SC',
    'sg_uf_residencia_SE',
    'sg_uf_residencia_SP',
    'sg_uf_residencia_TO',
    # 'sg_uf_residencia_Não informado',
]

In [21]:
new_features = X_train.columns.difference(unimportant_features + irrelevant_features)
len(new_features)

54

In [29]:
from sklearn.preprocessing import StandardScaler

regressor2 = make_pipeline(
    StandardScaler(),
    MultiOutputRegressor(
        Lasso(random_state=7),
        n_jobs=-1
    ),
)

regressor2.fit(X_train[new_features], y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('multioutputregressor',
                 MultiOutputRegressor(estimator=Lasso(random_state=7),
                                      n_jobs=-1))])

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

y_pred = regressor2.predict(X_test[new_features])
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R^2:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', sqrt(mean_squared_error(y_test, y_pred)))

MAE: 68.9939089710235
R^2: 0.25277103486872055
MSE: 9429.707781254672
RMSE: 97.10668247476417


In [34]:
df[new_features.to_list() + target_columns].to_csv('data/prep_train.csv', index=False)

## Treino sobre a base completa

In [35]:
X_train, y_train = df[new_features], df[target_columns]

In [37]:
regressorf = make_pipeline(
    StandardScaler(),
    MultiOutputRegressor(
        Lasso(random_state=7),
        n_jobs=-1
    ),
)

regressorf.fit(X_train[new_features], y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('multioutputregressor',
                 MultiOutputRegressor(estimator=Lasso(random_state=7),
                                      n_jobs=-1))])

In [39]:
import joblib
joblib.dump(regressorf, 'models/1_regressor.pkl')

['models/regressor.pkl']